# Multiplicación de Matrices con un `kernel` en bloques: una Introducción a programación en bloques (Tiled Programmation)

En esta sección nos dedicaremos a introducir un método de programación mucho más eficiente. En inglés es llamada *tiled programmation* y no hay una traducción exacta en español. Por eso la llamaremos *programación en bloques*, aunque he de admitir que es un pésimo nombre debido a que se presta a confundir los bloques de información, `tiles`, con los bloques de hilos, `thread blocks` o simplemente `blocks`.
Por lo que de ahora en adelante habrá de recodar que cuando se habla de programación en bloques, estos últimos se refieren a bloques de **información**.

En el ejemplo concreto del anterior `kernel` de multiplicación de matrices, cada dato de las dos matrices se encuentra en la `memoria global` y es utilizado por el `kernel` en distintas ocasiones. La idea de la programación en bloques es reducir y ordenar el número de accesos a los datos con ayuda de los distintos tipos de memoria que se encuentran en el GPU. Esto se observa mejor en las figuras siguientes.

<img src=Imagenes/Global.png align=center> 
<h4 align=center >La ilustración superior muestra el proceso cuando no programamos en bloque. En las imágenes inferiores pasamos a un proceso por `tiles`.</h4> 
<img src=Imagenes/Tiled.png align=center>
<h4 align=center >Fase 1 de la programación en bloques. En este caso la memoria `on-chip` equivale a la `shared memory`</h4>
<img src=Imagenes/Tiled2.png align=center>
<h4 align=center >Fase 2 de la programación en bloques.</h4>

Supongamos entonces que los `threads` 1 y 2 utilizan 12 datos guardados en la memoria **global**. Para optimizar los cálculos, estos los haremos en 3 fases.
En la primera guardamos sólo 4 datos en un bloque en la memoria **compartida**. Ahora los threads 1 y 2 tendrán un acceso más rápido a los datos que si estos estuvieran en la memoria global. Sin embargo hay que recordar que la memoria compartida tiene un tiempo de vida igual al `thread block`, por lo que cuando los `threads` 1 y 2 terminen con los 4 datos, estos serán eliminados de la memoria compartida. 
Ahora entramos en la fase 2, en la que guardamos los siguientes 4 datos en la memoria compartida. repitiéndose lo mismo que en la fase 1.
Así hasta la fase 3. 

**Siguiendo la analogía...**

Pensemos ahora que cada uno de los niños ya están en su casa después de un viaje en el camión escolar y están trabajando duramente en sus tareas, sin embargo ahora son sus padres quienes se preocupan en lo que prepararán para cenar, por lo que tienen que ir al super mercado para comprar los ingredientes con los que se preparará la comida.
En muchas ciudades del mundo, la distribución de algunos productos se encuentra centralizado en algunos barrios de la ciudad, por lo que encontrar algunos productos requiere de viajar grandes distancias hasta el lugar en el que se encuentra el producto deseado.
Sin embargo desde hace décadas algunas otras ciudades han implementado políticas para descentralizar estos mercados, por lo que si una familia vive en algún barrio suburbano, esta no tenga que viajar hasta el centro de la ciudad para conseguir algunos productos. Esto le permite a las personas optimizar sus viajes.

La idea de la programación en bloques es la misma. Los `threads` necesitan operandos (comida) para trabajar. Al programar en bloque uno lleva la comida al super mercado local en el que se encuentra el `thread` de tal manera que este no tenga que viajar al centro de la ciudad. 

Sin embargo podemos encontrar dos distintos tipos de problemas que pueden presentarse en estos casos. 
Uno es cuando en el barrio local, cada una de las familias que viven ahí no buscan los mismos productos, es decir que en un `thread block`, cada uno de estos no busca los mismos operandos. 
Otro es cuando el supermercado local no tiene la suficiente infraestructura para darle cábida a toda la comida necesaria, i.e. no hay el suficiente espacio en la `memoria compartida` para traer todos los operandos necesarios.

En el primer caso, hay que encontrar una serie de datos la cual sea utilizada por múltiples `threads`, asegurándonos que cada dato será utilizado para luego pasar a la siguiente serie de datos, una vez que los `threads` hayan hecho sus operaciones.

El segundo caso tiene que ver con la capacidad de la memoria compartida. Podemos darle a esta dos configuraciones:
16 KB de memoria compartida o 48KB en el `cache L1`. Esto nos permite dar de nuevo los detalles a la hora de elegir el tamaño del `bloque`, o viceversa. Tomemos aquella con 16KB de memoria compartida.

+ Supongamos que elegimos de nuevo un tamaño de `thread block` de 16x16. Puesto que cada `thread` carga dos datos (uno de la matriz $A$ y otro de la matriz $B$) y cada uno de estos pesa 4B, entonces obtenemos que cada uno de los bloques utiliza 256x2x4B = 2KB, por lo que podremos utilizar los 8 bloques permitidos para llenar los espacios permitidos en el `SM`.
+ Si ahora utilizamos un `bloque` de 32x32, entonces cada bloque usará 1024x2x4B = 8KB, por lo que únicamente podremos utilizar 2 bloques. Sin embargo, cada uno de los bloques tiene 1024 `threads`, resultando que sólo podremos utilizar uno debido a la restricción de nuesto `SM` de almacenar 1536 `threads` como máximo.

De esta manera la dimensión del bloque y el número de operaciones hechas por los `threads` son a considerarse a la hora de utilizar la programación en bloques. 

**Después de todas estas restricciones y detalles en los que hay que pensar, uno llega a preguntarse: ¿realmente vale la pena este tipo de programación?**

Esto puede contestarse en parte por los siguientes datos:

En general un `GPU` tiene un máximo de 1000GFLOPS.
En nuestro kernel básico de multiplicación de matrices, cada `thread` utiliza 4B por cada dato, por lo que si buscamos tener 1 FLOPS, entonces necesitamos una velocidad de 4B/s, por lo que si buscamos maximizar la velocidad de nuestro código, entonces estamos esperando un ancho de banda de 1000 operaciones x 4B/s = 4000GB/s.
Ahora bien, con un `GPU` estándar como en el que se corrió este código tenemos un ancho de banda entre los threads y la memoria global de 150GB/s, dándonos un total de 37.5GFLOPS cuando dividimos esta entre 4. Sin embargo a la hora de correr el código obtenemos un total de aproximadamente 25GFLOPS. 

En el caso de nuestro kernel con programación en bloque, el hecho de poner como intermediario a la `memoria compartida` provoca que el acceso a la `memoria global` descienda por un factor de 16 en el caso de los `blocks` de 16x16 y de 32 en el caso de 32x32. Haciendo de nuevo los cálculos para el caso de 16x16 (el cual ya vimos que es el más óptimo) tenemos entonces que el número máximo de GFLOPS al que podremos llegar es (150/4)16 = 600 GFLOPS!!!! Este número es mucho más cercano al máximo de nuestro `GPU`. 

Esto es únicamente una pequeña discusión sobre los beneficios de nuestro código en bloques, sin embargo es importante ver e interactuar con los resultados.

Lo primero que tenemos que hacer en nuestro código será declarar los `bloques de información` o (`tiles`) en la `memoria compartida`. Esto se hace de la siguiente manera:

```C

#define TILE_WIDTH 16

__shared__ float ds_A[TILE_WIDTH][TILE_WIDTH] ;
__shared__ float ds_B[TILE_WIDTH][TILE_WIDTH] ;

```

Se define `TILE_WIDTH` afuera del `kernel`, pero aquí lo ponemos para fines demostrativos. Lo que hace `TILE_WIDTH` es fijar la dimensión de nuestros `bloques de información`, que en este caso será 16 por la discusión anterior sobre la eficiencia de nuestro código.

Un detalle es notar cómo llamamos los `bloques de información`. el sufijo `ds_` se refiere a que vive en el `device` o `GPU`.

Para llenar la memoria compartida de los datos de nuestra matriz, tendremos que usar un bucle `for` con el que podamos pasar por todos los `thread blocks` que se tienen.

```C

for (int t = 0 ; t < (numColA - 1)/TILE_WIDTH + 1 ; t ++) 
{

}

```

donde `t` es el número de `bloque`. Es importante notar la expresión `(numColA - 1)/TILE_WIDTH + 1`, que como resultado nos da el número de separaciones que la matriz puede tener con la dimensión de los `bloques de información`.
Dicho de otro modo, el parámetro `t` utilizado en el `for` nos da el número de `tile` en el cual estamos trabajando. El número máximo de bloques o `tiles` con los que trabajaremos está dado, en el caso de una multiplicación de matrices de $m\times n$ y $n\times k$ por `(n-1)/TILE_WIDTH + 1`; algo así como el algoritmo que utilizamos en el capítulo anterior para saber la dimensión de nuestra `grid`.

**Nota:** Esto nos da una buena idea de las dimensiones de la `malla` y de los `bloques` que luego se tendrán que fijar en el código.

Dentro de este `for` habrá que llenar la memoria compartida. Esto se hace con un código de la forma

```C
if (Fila < numFilasA && t*TILE_WIDTH + tx < numColA) 
{
    ds_A[ty][tx] = A[Row*numColA + t*TILE_WIDTH +tx] ;
} else {
    ds_A[ty][tx] = 0.0 ;
}
```

Los dos primeros condicionales `if/else` son condiciones de frontera y nos permiten almacenar la cantidad exacta de datos. Supongamos que nuestra matriz $A$ es de 256x257, entonces necesitaremos de 17 `tiles` en la dirección x, haciendo que 12 `threads` no tengan asignados ningún valor de la matriz $A$. Para evitar que tengan información no deseada, hacemos a todos estos sobrantes 0.0. Pasa lo mismo para la matriz $B$.
El índice utilizado en la matriz `A` es un poco más complicado que todos aquellos índices vistos hasta ahora, sin embargo le dejaremos al lector su comprensión.

En el caso de `ds_B`, el código es el siguiente:

```C

if (t*TILE_WIDTH + ty < numColA && Col < numColB)
{
    ds_B[ty][tx] = B[(t*TILE_WIDTH +ty)*numColB + Col] ;
} else {
    ds_B[ty][tx] = 0.0 ;
}

```
Recuerda, a la hora de escribir tu código, de colocar un `__syncthreads()` al final de estos dos párrafos para evitar cálculos equivocados.

Finalmente calculamos las entradas de la nueva matriz.

```C

for (int i=0; i < TILE_WIDTH; i ++) 
{
    Cvalue += ds_A[ty][i] * ds_B [i][tx] ;
}

__syncthreads() ;
if (Fila<numFilasA && Col<numColB) 
{
    C[Fila*numColB + Col] = Cvalue ;
}
```

la variable `Cvalue` está definida afuera de los condicionales y los loops. Esto podemos hacerlo gracias al uso de `tiles`.

Sin más, ¡a escribir se ha dicho!

In [ ]:
#define TILE_WIDTH

// Escribir el kernel


int main( int argc, char * argv[])
{
    
    int numColA 1000 ;
    int numColB 1000 ;
    int numColC 1000 ;
    
    int numFilaA 1000 ;
    int numFilaB 1000 ;
    int numFilaC 1000 ;
    
    float A[numFilaA*numColA] ;
    float B[numFilaB*numColB] ;
    float C[numFilaC*numColC] ;
    
    for (i = 0 ; i < numFilaA*numColA ; i ++)
    {
        A[i] = i ;
    }

    for (i = 0 ; i < numFilaB*numColB ; i ++)
    {
        B[i] = i + 1 ;
    }
    
    // Escribe abajo las lineas para la alocacion de memoria
    
    // Escribe abajo las lineas para la copia de memoria del CPU al GPU
    
    // Escribe las dimensiones de los bloques y mallas
    
    // Escribe las lineas para lanzar el kernel
    
    // Copia la memoria del GPU al CPU
      
    // Libera la memoria
      
    // Escribe un codigo para poder saber si tu resultado es correcto
    
    
    return 0 ;
}

Probado localmente, se utilizaron dos matrices con dimensiones 200x100 y 100x256. Al correr nuestro código con el `kernel` básico, el cálculo se realizó en 0.24 ms.
Cuando se compiló el código con un `kernel` en bloques o `tiled kernel`, el computo se realizó en  ¡¡¡0.13ms!!!.

Se invita al lector a que implemente funciones para comparar los tiempos entre los dos kernels y probar la eficacia de la **tiled programmation**. 

También se invita al lector a jugar con las dimensiones de los `bloques de información` para mostrar cómo cambia la eficacia del código con respecto a este parámetro.

## Conclusiones generales de CUDA C

En esta primera parte nos hemos apoyado de ejemplos clásicos para poder mostrar las bases de la programación en CUDA C. A partir de aquí quedan todavía una enorme cantidad de detalles por ver, sin embargo consideramos que el lector, para los fines de este curso, podrá continuar sin mayor problema. 

En el siguiente notebook hemos preparado una serie de ejercicios para poder seguir con el entrenamiento de CUDA C. También, para ir más lejos en algunos tópicos invitamos a consultar los libros puestos en las referencias. 

CUDA C, aunque parezca tedioso tiene un gran número de ventajas. Esperamos que hasta este momento el lector se haya ya convencido de todo aquello que se puede hacer con la computación en paralelo y que decida continuar con nosotros para ahora empezar con cosas más interesantes. 

En la segunda parte usaremos la física estadística como pretexto para aprender aún más de CUDA C y de pyCUDA, e inversamente nos dedicaremos a enseñar un poco de física estadística usando las ventajas de CUDA C y pyCUDA.

